In [10]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: myLearning
Azure region: westus2
Subscription id: e1cf48d1-ba12-4ed0-b31a-84a953eee16d
Resource group: Obi-ML


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# CPU cluster name
amlcompute_cluster_name = "ccluster-exer01"

# requisite from the rubric: Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Cluster already exist.')
except ComputeTargetException:
    print('Creating requested computer cluster, please wait...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Cluster already exist.

Running


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
#libraries required for the Param Sampling
from azureml.train.hyperdrive import choice, loguniform
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C': uniform(0.05, 1),
        '--max_iter': choice(20, 40, 60, 80, 100)
    }
)

# Specify a Policy
### YOUR CODE HERE: Early termination policy ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
training_folder = "./training"
est = SKLearn(source_directory=training_folder, entry_script = "train.py", compute_target = aml_compute)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=50,
                                    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
### YOUR CODE HERE ###
hdr = exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_74103b9b-0c25-4caa-8095-dd70ec1146e3
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_74103b9b-0c25-4caa-8095-dd70ec1146e3?wsid=/subscriptions/e1cf48d1-ba12-4ed0-b31a-84a953eee16d/resourcegroups/Obi-ML/workspaces/myLearning

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-15T23:59:49.753564][API][INFO]Experiment created<END>\n""<START>[2021-01-15T23:59:50.336910][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-01-15T23:59:50.5635318Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-15T23:59:50.706624][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


In [11]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hdr.get_best_run_by_primary_metric()
parameter_values = best_run.get_details()['runDefinition']['arguments']
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:', parameter_values[3])

#registering the model
best_model = best_run.register_model(model_name='bankmarketing-hyper_model', model_path='azureml-logs/bmhypermodel.pkl')

Best Run Id:  HD_74103b9b-0c25-4caa-8095-dd70ec1146e3_0

 Accuracy: 0.9146181082448154

 learning rate: 40


In [41]:
print(parameter_values)

['--C', '0.341258367683086', '--max_iter', '40']


In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
web_path = [
       'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
   ]
dsbmarketing = TabularDatasetFactory.from_delimited_files(path=web_path, separator=',')

In [5]:
dsb = dsbmarketing.to_pandas_dataframe()
dsb.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [6]:
dsb.education=='unknown'

0        False
1         True
2        False
3        False
4        False
         ...  
32945    False
32946    False
32947    False
32948    False
32949    False
Name: education, Length: 32950, dtype: bool

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
sys.path.append("training")
from train import clean_data

# Use the clean_data function to clean your data.
#x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(dsbmarketing)
xds_train, xds_test, yds_train, yds_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
#automl_config = AutoMLConfig(
#    experiment_timeout_minutes=30,
#    task=,
#    primary_metric=,
#    training_data=,
#    label_column_name=,
#    n_cross_validations=)
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dsbmarketing,
    label_column_name='y',
    n_cross_validations=5,
    max_concurrent_iterations= 4,
    max_cores_per_iteration= -1,    
    iterations=50,    
    compute_target=aml_compute,
    enable_onnx_compatible_models=True
    )

automl_exp = Experiment(workspace=ws, name="udacity-automlproject")

In [15]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
aml_marketing_run = automl_exp.submit(automl_config, show_output = True)
RunDetails(aml_marketing_run).show()
aml_marketing_run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on ccluster-exer01 with default configuration
Running on remote compute: ccluster-exer01
Parent Run ID: AutoML_0fc96e52-25d3-48aa-8ee3-e4ce809cd7b2

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because t

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|3692                             |yes                              |32950                                 |
+---------------------------------+----------

{'runId': 'AutoML_0fc96e52-25d3-48aa-8ee3-e4ce809cd7b2',
 'target': 'ccluster-exer01',
 'status': 'Completed',
 'startTimeUtc': '2021-01-17T00:02:09.810801Z',
 'endTimeUtc': '2021-01-17T00:47:55.559438Z',
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ccluster-exer01',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-automlproject","subscription_id":"e1cf48d1-ba12-4ed0-b31a-84a953eee16d","resource_group":"Obi-ML","workspace_name":"myLearning","region":"westus2","compute_target":"ccluster-exer01","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":50,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization":"auto

In [16]:
best_run, best_model = aml_marketing_run.get_output()
vc = best_model.steps[1][1]
print(vc.named_estimators)

{'47': Pipeline(memory=None,
         steps=[('standardscalerwrapper',
                 <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f12520a9438>),
                ('xgboostclassifier',
                 XGBoostClassifier(base_score=0.5, booster='gbtree',
                                   colsample_bylevel=1, colsample_bynode=1,
                                   colsample_bytree=0.8, eta=0.3, gamma=0,
                                   learning_rate=0.1, max_delta_step=0,
                                   max_depth=6, max_leaves=31,
                                   min_child_weight=1, missing=nan,
                                   n_estimators=100, n_jobs=-1, nthread=None,
                                   objective='reg:logistic', random_state=0,
                                   reg_alpha=2.5, reg_lambda=1.0416666666666667,
                                   scale_pos_weight=1, seed=None, silent=None,
                                   subsa

In [17]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_aml_marketing_model = aml_marketing_run.get_output()
from azureml.automl.runtime.onnx_convert import OnnxConverter
best_aml_marketing_model, onnx_model= aml_marketing_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")
print(best_aml_marketing_model)

Run(Experiment: udacity-automlproject,
Id: AutoML_0fc96e52-25d3-48aa-8ee3-e4ce809cd7b2_49,
Type: azureml.scriptrun,
Status: Completed)


In [14]:
#once the experiments are completed, delete the cluster
aml_compute.delete()